# Single Pulse Response

This notebook launches a single NRZ pulse into the SerDes signal path and inspects the resulting response at the receiver.


In [ ]:
import sys, pathlib

repo_root = pathlib.Path.cwd()
while repo_root != repo_root.parent:
    if (repo_root / 'serdes').exists():
        break
    repo_root = repo_root.parent
else:
    raise RuntimeError("Could not locate repository root containing 'serdes'.")

repo_root_str = str(repo_root)
if repo_root_str not in sys.path:
    sys.path.insert(0, repo_root_str)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from serdes import ChannelConfig, RxEqualizerConfig, SignalConfig, TxEqualizerConfig
from serdes.channel import LinearChannel
from serdes.filters import RxContinuousTimeLinearEqualizer, TxFeedForwardEqualizer
from serdes.jitter import JitterInjector
from serdes.config import JitterConfig
from serdes.signal import pulse_shaping


In [ ]:
signal = SignalConfig(
    modulation='nrz',
    symbol_rate=32e9,
    samples_per_symbol=64,
    amplitude=1.0,
    pattern='prbs7',  # pattern is ignored because we provide explicit symbols
)
tx = TxEqualizerConfig(taps=(0.0, 1.0, 0.0))
channel = ChannelConfig(alpha=0.18, post_cursor=10)
rx = RxEqualizerConfig(ctle_zero_hz=12e9, ctle_pole_hz=32e9, ctle_dc_gain=1.0)
jitter = JitterInjector(JitterConfig(enable=False))

num_symbols = 128
symbols = np.zeros(num_symbols)
pulse_index = num_symbols // 2
symbols[pulse_index] = signal.amplitude

waveform = pulse_shaping(symbols, signal)
ffe = TxFeedForwardEqualizer(tx)
waveform_tx = ffe.apply(waveform)
channel_model = LinearChannel(channel, signal.samples_per_symbol)
waveform_channel = channel_model.apply(waveform_tx)
ctle = RxContinuousTimeLinearEqualizer(rx, signal)
waveform_rx = ctle.apply(waveform_channel)
waveform_rx = jitter.apply(waveform_rx)


In [ ]:
samples_per_symbol = signal.samples_per_symbol
center_sample = pulse_index * samples_per_symbol + samples_per_symbol // 2
time_window = slice(max(center_sample - 4 * samples_per_symbol, 0),
                     min(center_sample + 4 * samples_per_symbol, len(waveform_rx.samples)))

fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(waveform_tx.time[time_window] * 1e9, waveform_tx.samples[time_window], label='TX pulse')
ax.plot(waveform_rx.time[time_window] * 1e9, waveform_rx.samples[time_window], label='RX response')
ax.axvline(waveform_rx.time[center_sample] * 1e9, color='k', linestyle='--', label='Decision point')
ax.set_xlabel('Time (ns)')
ax.set_ylabel('Amplitude (V)')
ax.set_title('Single Pulse Response Through the Link')
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
rx_symbol = waveform_rx.samples[center_sample]
print(f'Center sample amplitude: {rx_symbol:.3f} V')
